# cargar los datos 

In [41]:
# Importamos las librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer


# Cargamos los datos
valoraciones = pd.read_json('data/web_reviews.json')

# Exploración de los datos

In [42]:
# Visualizamos las primeras filas del dataset
print("Primeras filas del dataset:")
print(valoraciones.head())

# Revisamos la estructura de las columnas y el tipo de datos
print("\nInformación del dataset:")
print(valoraciones.info())

# Calculamos estadísticas descriptivas para la columna 'overall'
print("\nEstadísticas descriptivas de la calificación 'overall':")
print(valoraciones['overall'].describe())

# Revisión de valores únicos y su distribución en 'overall'
print("\nDistribución de calificaciones (overall):")
print(valoraciones['overall'].value_counts())

# Análisis de la longitud de los textos en 'reviewText' para entender el contenido
valoraciones['review_length'] = valoraciones['reviewText'].apply(len)
print("\nLongitud promedio de las reseñas:")
print(valoraciones['review_length'].mean())

Primeras filas del dataset:
       reviewerID        asin  \
0  A2IBPI20UZIR0U  1384719342   
1  A14VAT5EAX3D9S  1384719342   
2  A195EZSQDW3E21  1384719342   
3  A2C00NNG1ZQQG2  1384719342   
4   A94QU4C90B1AX  1384719342   

                                       reviewerName   helpful  \
0  cassandra tu "Yeah, well, that's just like, u...    [0, 0]   
1                                              Jake  [13, 14]   
2                      Rick Bennette "Rick Bennette    [1, 1]   
3                          RustyBill "Sunday Rocker    [0, 0]   
4                                     SEAN MASLANKA    [0, 0]   

                                          reviewText  overall  \
0  Not much to write about here, but it does exac...        5   
1  The product does exactly as it should and is q...        5   
2  The primary job of this device is to block the...        5   
3  Nice windscreen protects my MXL mic and preven...        5   
4  This pop filter is great. It looks and perform...     

## entrenamiento 

In [43]:
# Eliminación de valores nulos en 'overall' y 'reviewText'
valoraciones = valoraciones.dropna(subset=['overall', 'reviewText'])

# Convertir 'overall' en categorías: 1-2 = negativa, 3 = neutral, 4-5 = positiva
def categorize_rating(rating):
    if rating <= 2:
        return "negativa"
    elif rating == 3:
        return "neutral"
    else:
        return "positiva"

valoraciones['sentimiento'] = valoraciones['overall'].apply(categorize_rating)

# Definimos el objetivo (target) y la característica (feature)
y = valoraciones['sentimiento']

# Convertir el texto de 'reviewText' en características numéricas usando TF-IDF
tfidf = TfidfVectorizer(max_features=100)
X_text = tfidf.fit_transform(valoraciones['reviewText']).toarray()

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size=0.2, random_state=0)

# Crear el modelo de clasificación
model = RandomForestClassifier(n_estimators=100, random_state=0)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones
predictions = model.predict(X_test)

# Evaluar el modelo con accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"\nAccuracy: {accuracy}")

# También se puede calcular el MAE como referencia
mae = mean_absolute_error(y_test.map({'negativa': 0, 'neutral': 1, 'positiva': 2}), 
                          pd.Series(predictions).map({'negativa': 0, 'neutral': 1, 'positiva': 2}))
print(f"\nError absoluto medio (MAE) para las categorías: {mae}")


Accuracy: 0.8714076960545543

Error absoluto medio (MAE) para las categorías: 0.19581100828056502


## Tratamiento de valores faltantes

In [52]:
# Verificar valores faltantes en todas las características del DataFrame 'valoraciones'
print("\nValores faltantes en cada característica:")
print(valoraciones.isnull().sum())




Valores faltantes en cada característica:
reviewerID        0
asin              0
reviewerName      0
helpful           0
reviewText        0
overall           0
summary           0
unixReviewTime    0
reviewTime        0
instrumentType    0
review_length     0
sentimiento       0
dtype: int64


## Imputación

In [45]:
# Convertir el texto de 'reviewText' en características numéricas usando TF-IDF
tfidf = TfidfVectorizer(max_features=100)
X_text = tfidf.fit_transform(valoraciones['reviewText']).toarray()

# Lista para guardar resultados
results = []

# Función para evaluar el modelo y registrar los resultados
def evaluate_model(X_train, X_test, y_train, y_test, technique):
    # Crear el modelo de clasificación
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    # Calcular accuracy y MAE
    accuracy = accuracy_score(y_test, predictions)
    mae = mean_absolute_error(y_test.map({'negativa': 0, 'neutral': 1, 'positiva': 2}), 
                              pd.Series(predictions).map({'negativa': 0, 'neutral': 1, 'positiva': 2}))
    
    # Guardar los resultados
    results.append({"Technique": technique, "Accuracy": accuracy, "MAE": mae})

# Separar las características y el target
X = X_text
y = valoraciones['sentimiento']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Imputación con extensión (Media)

In [46]:
# 1. Imputación con la media
imputer_mean = SimpleImputer(strategy='mean')
X_train_mean = imputer_mean.fit_transform(X_train)
X_test_mean = imputer_mean.transform(X_test)
evaluate_model(X_train_mean, X_test_mean, y_train, y_test, "Imputación con Media")

## Imputación con extensión (Mediana)

In [47]:
# 2. Imputación con la mediana
imputer_median = SimpleImputer(strategy='median')
X_train_median = imputer_median.fit_transform(X_train)
X_test_median = imputer_median.transform(X_test)
evaluate_model(X_train_median, X_test_median, y_train, y_test, "Imputación con Mediana")


## Imputación con extension (Moda)

In [48]:

# 3. Imputación con la moda
imputer_moda = SimpleImputer(strategy='most_frequent')
X_train_moda = imputer_moda.fit_transform(X_train)
X_test_moda = imputer_moda.transform(X_test)
evaluate_model(X_train_moda, X_test_moda, y_train, y_test, "Imputación con Moda")


## Inputacion con extensión (Valor constante)

In [49]:

# 4. Imputación con un valor constante (ej. 0)
imputer_constant = SimpleImputer(strategy='constant', fill_value=0)
X_train_constant = imputer_constant.fit_transform(X_train)
X_test_constant = imputer_constant.transform(X_test)
evaluate_model(X_train_constant, X_test_constant, y_train, y_test, "Imputación con Valor Constante (0)")

## Eliminación 


In [50]:

# 5. Eliminación de valores nulos
# Convertimos X_train y X_test a DataFrames para permitir la manipulación de índices
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)
# Eliminación de valores nulos y ajuste de índices
X_train_dropped = X_train_df.dropna().reset_index(drop=True)
y_train_dropped = y_train.iloc[X_train_dropped.index].reset_index(drop=True)
X_test_dropped = X_test_df.dropna().reset_index(drop=True)
y_test_dropped = y_test.iloc[X_test_dropped.index].reset_index(drop=True)

# Evaluar modelo con eliminación de valores nulos
evaluate_model(X_train_dropped.values, X_test_dropped.values, y_train_dropped, y_test_dropped, "Eliminación de Valores Nulos")

## Justificación 
- Eliminación de Filas: Eliminamos filas con reviewText nulo, ya que sin texto, la reseña no aporta al análisis de sentimientos.
- Imputación con Media o Mediana: Útil para características numéricas en las que deseamos preservar el promedio o el valor central del dataset.
- Imputación con Moda: Apropiada para características categóricas, ya que reemplaza valores faltantes con el valor más común en la columna.
- Imputación con Valor Constante: Indicada para características donde el valor nulo puede ser interpretado como "ausencia de valor" (por ejemplo, 0 podría representar la falta de un atributo en algunas características)

In [51]:
# Mostrar los resultados
results_df = pd.DataFrame(results)
print(results_df)

                            Technique  Accuracy       MAE
0                Imputación con Media  0.871408  0.195811
1              Imputación con Mediana  0.871408  0.195811
2                 Imputación con Moda  0.871408  0.195811
3  Imputación con Valor Constante (0)  0.871408  0.195811
4        Eliminación de Valores Nulos  0.871408  0.195811
